In [77]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import folium
from math import radians, sin, cos, sqrt, atan2

In [ ]:
# Endpoint GraphDB
GRAPHDB_ENDPOINT = "http://localhost:7200/repositories/Fil_Rouge_DB"

In [79]:
# Fonction pour exécuter une requête SPARQL
def run_sparql_query(query):
    sparql = SPARQLWrapper(GRAPHDB_ENDPOINT)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    sparql.addParameter("sameAs", "false")
    results = sparql.query().convert()
    return results

In [80]:
# Fonction pour calculer la distance entre deux points en utilisant la formule de Haversine
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convertir les degrés en radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    # Rayon de la Terre en km
    km = 6371 * c
    return km * 1000  # conversion en mètres

In [81]:
# Pour chaque ville, compter le nombre d'organisations dans un rayon de 20 km (20 000 mètres)
def count_orgs_within_radius(city_lat, city_lon, city_country, orgs_df, radius=20000):
    # Filtrer uniquement les organisations du même pays
    orgs_in_country = orgs_df[orgs_df['country'] == city_country]
    distances = orgs_in_country.apply(lambda row: haversine_distance(city_lat, city_lon, row["lat"], row["lon"]), axis=1)
    return (distances <= radius).sum()

In [82]:
orgs_data = [
    {"org": "Marseille", "lat": 43.2965, "lon": 5.3698, "country": "France"},
    {"org": "Nanterre",  "lat": 48.8924, "lon": 2.1969, "country": "France"}
]
orgs_df = pd.DataFrame(orgs_data)

# Test pour la ville de Paris
paris_lat = 48.8566
paris_lon = 2.3522
paris_country = "France"

# Calcul du nombre d'organisations dans un rayon de 20 km autour de Paris
org_count = count_orgs_within_radius(paris_lat, paris_lon, paris_country, orgs_df, radius=20000)

print("Nombre d'organisations à moins de 20 km de Paris :", org_count)

Nombre d'organisations à moins de 20 km de Paris : 1


In [83]:
# Requête SPARQL pour récupérer les villes
cities_query = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX ns1: <http://www.semanticweb.org/fil_rouge_AntoineValette_GuillaumeRamirez/>

SELECT ?city ?cityName ?lat ?lon ?population ?country
WHERE {
  ?city a ns1:City .
  ?city ns1:population ?population .
  ?city skos:prefLabel ?cityName .
  ?city ns1:Latitude ?lat .
  ?city ns1:Longitude ?lon .
  ?city ns1:aPourPays ?country
  FILTER(?population >= 300000)
}
"""

In [84]:
# Requête SPARQL pour récupérer les organisations
orgs_query = """
PREFIX ns1: <http://www.semanticweb.org/fil_rouge_AntoineValette_GuillaumeRamirez/>

select ?org ?lat ?lon ?country where {
    ?org a ns1:Organization .
    ?org ns1:inCity ?city .
    ?city ns1:Latitude ?lat .
    ?city ns1:Longitude ?lon .
    ?org ns1:inCountry ?country
}
"""

In [85]:
# Récupération des données des villes
cities_results = run_sparql_query(cities_query)
cities_data = []
for result in cities_results["results"]["bindings"]:
    cityName = result["cityName"]["value"]
    lat = float(result["lat"]["value"])
    lon = float(result["lon"]["value"])
    population = float(result["population"]["value"])
    country = result["country"]["value"]
    cities_data.append({
        "city": cityName,
        "lat": lat,
        "lon": lon,
        "population": population,
        "country": country
    })

In [86]:
cities_df = pd.DataFrame(cities_data)

In [87]:
cities_df.shape

(1814, 5)

In [88]:
# Récupération des données des organisations
orgs_results = run_sparql_query(orgs_query)
orgs_data = []
for result in orgs_results["results"]["bindings"]:
    org = result["org"]["value"]
    lat = float(result["lat"]["value"])
    lon = float(result["lon"]["value"])
    country = result["country"]["value"]
    orgs_data.append({
        "org": org,
        "lat": lat,
        "lon": lon,
        "country": country
    })

In [89]:
orgs_df = pd.DataFrame(orgs_data)

In [90]:
orgs_df.head()

,org,lat,lon,country
0,https://www.crunchbase.com/organization/0023af...,45.52345,-122.67621,https://www.geonames.org/countries/US
1,https://www.crunchbase.com/organization/002c5c...,37.56299,-122.32553,https://www.geonames.org/countries/US
2,https://www.crunchbase.com/organization/00e435...,47.60621,-122.33207,https://www.geonames.org/countries/US
3,https://www.crunchbase.com/organization/018d81...,37.83132,-122.28525,https://www.geonames.org/countries/US
4,https://www.crunchbase.com/organization/0209db...,-34.61315,-58.37723,https://www.geonames.org/countries/AR


In [ ]:
# Appliquer le calcul pour chaque ville
cities_df["org_count"] = cities_df.apply(
    lambda row: count_orgs_within_radius(row["lat"], row["lon"], row["country"], orgs_df),
    axis=1
)

               city       lat        lon  population  \
160           Brent  51.55306   -0.30230    329100.0   
1785         London  51.50853   -0.12574   8961989.0   
114       Islington  51.53622   -0.10304    319143.0   
1200  San Francisco  37.77493 -122.41942    864816.0   
507         Oakland  37.80437 -122.27080    419267.0   

                                    country  org_count  
160   https://www.geonames.org/countries/GB      12387  
1785  https://www.geonames.org/countries/GB      12348  
114   https://www.geonames.org/countries/GB      12322  
1200  https://www.geonames.org/countries/US      11875  
507   https://www.geonames.org/countries/US      11568  


In [96]:
# Sélection des 5 villes ayant le plus d'organisations dans un rayon de 20 km,
# tout en s'assurant qu'elles soient séparées d'au moins 20 km les unes des autres.

selected_cities = []
sorted_cities = cities_df.sort_values(by="org_count", ascending=False)

for idx, row in sorted_cities.iterrows():
    too_close = False
    # Vérifier la distance entre la ville candidate et chacune des villes sélectionnées
    for selected in selected_cities:
        distance = haversine_distance(row["lat"], row["lon"], selected["lat"], selected["lon"])
        if distance < 20000:
            too_close = True
            break
    if not too_close:
        selected_cities.append(row)
    # Arrêter dès qu'on a 30 villes
    if len(selected_cities) >= 30:
        break

# Convertir la liste en DataFrame pour traitement ultérieur
top_cities = pd.DataFrame(selected_cities)
print(top_cities)

               city       lat        lon  population  \
160           Brent  51.55306   -0.30230    329100.0   
1200  San Francisco  37.77493 -122.41942    864816.0   
1732    Los Angeles  34.05223 -118.24368   3898747.0   
1634          Paris  48.85341    2.34880   2138551.0   
1096        Seattle  47.60621 -122.33207    737015.0   
1682        Chicago  41.85003  -87.65005   2696555.0   
332       Etobicoke  43.64415  -79.56985    365000.0   
36          Vaughan  43.83610  -79.49827    306233.0   
728      Karol Bāgh  28.65136   77.19072    505241.0   
1475      San Diego  32.71571 -117.16472   1394928.0   
897        Itabashi  35.74893  139.71497    584483.0   
1524       Kawasaki  35.52056  139.71722   1538262.0   
1782      Bengaluru  12.97194   77.59369   8443675.0   
1710         Madrid  40.41650   -3.70256   3255944.0   
1308         Dublin  53.33306   -6.24889   1024027.0   
554        Tel Aviv  32.08088   34.78057    432892.0   
194       Santa Ana  33.74557 -117.86783    3354

In [97]:
# --- Visualisation avec Folium ---

# Créer la carte centrée sur (0, 0) avec un zoom arrière (par exemple, 2) et un fond de carte différent
m = folium.Map(location=[0, 0], zoom_start=2, tiles="CartoDB positron")

for idx, row in top_cities.iterrows():
    # Marker sous forme de CircleMarker pour une meilleure esthétique
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=8,  # taille du marker
        popup=folium.Popup(f"{row['city']} : {row['org_count']} organisations", parse_html=True),
        color="darkblue",
        fill=True,
        fill_color="lightblue",
        fill_opacity=0.7
    ).add_to(m)
    
    # Cercle de 20 km autour de la ville, avec un style plus discret
    folium.Circle(
        location=[row["lat"], row["lon"]],
        radius=20000,  # 20 km en mètres
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.1,
        weight=2
    ).add_to(m)

# Afficher la carte dans le notebook
m